# Alternative names (combinations) for top authors

In [1]:
import csv
import os
import re
from collections import defaultdict
from collections import Counter
from datetime import datetime
from datetime import timedelta
from string import punctuation

import matplotlib
import numpy as np
import networkx as nx
import pandas as pd
import requests
import seaborn as sns
from matplotlib import pyplot as plt
from SPARQLWrapper import SPARQLWrapper, JSON

# Autores

In [2]:
%%time

dtypes = {
    'TITULO': str,
    'AUTOR': str,
    'PAIS_EJEMPLAR': str,
    'PAIS_AUTOR': str,
}

autores_df = pd.read_csv('autores3.csv', header=0, dtype=dtypes)
autores_df = autores_df.fillna('')

CPU times: user 4.69 s, sys: 272 ms, total: 4.97 s
Wall time: 4.96 s


In [3]:
autores_df.shape

(3054360, 4)

In [4]:
autores_df.head()

,TITULO,PAIS_EJEMPLAR,AUTOR,PAIS_AUTOR
0,Un bicho extraño,Spain,Mon Daporta,DESCONOCIDO
1,Un bicho extraño,Spain,Mon Daporta,DESCONOCIDO
2,Quisiera tener ...,Mexico,Giovanna Zoboli,DESCONOCIDO
3,Johannes Gutenberg,Spain,Lluís Borràs Perelló,DESCONOCIDO
4,La fantástica leyenda de: la princesa y el dragón,Spain,Sonia Alins,DESCONOCIDO


# Autores con país desconocido

In [5]:
autores_pais_desconocido_df = autores_df[
    (autores_df['AUTOR'] != 'DESCONOCIDO') &
    (autores_df['PAIS_AUTOR'] == 'DESCONOCIDO')
]

In [6]:
autores_pais_desconocido_df.shape

(1643766, 4)

In [7]:
autores_pais_desconocido_df.head()

,TITULO,PAIS_EJEMPLAR,AUTOR,PAIS_AUTOR
0,Un bicho extraño,Spain,Mon Daporta,DESCONOCIDO
1,Un bicho extraño,Spain,Mon Daporta,DESCONOCIDO
2,Quisiera tener ...,Mexico,Giovanna Zoboli,DESCONOCIDO
3,Johannes Gutenberg,Spain,Lluís Borràs Perelló,DESCONOCIDO
4,La fantástica leyenda de: la princesa y el dragón,Spain,Sonia Alins,DESCONOCIDO


In [8]:
count_autores_desconocidos_sr = \
    autores_pais_desconocido_df.groupby(['AUTOR'])['TITULO'].count().sort_values(ascending=False)

In [9]:
count_autores_desconocidos_sr.head()

AUTOR
Colombia                                                   11526
Friedrich Wilhelm Nietzsche                                 5876
Ivar Da Coll                                                5350
Keiko Kasza                                                 5336
Instituto Colombiano de Normas Técnicas y Certificación     4333
Name: TITULO, dtype: int64

In [10]:
THRESHOLD = 500

In [11]:
top_autores_desconocidos_sr = count_autores_desconocidos_sr[lambda x: x >= THRESHOLD]

In [12]:
top_autores_desconocidos_sr.shape

(277,)

In [13]:
top_autores_desconocidos_sr

AUTOR
Colombia                                                   11526
Friedrich Wilhelm Nietzsche                                 5876
Ivar Da Coll                                                5350
Keiko Kasza                                                 5336
Instituto Colombiano de Normas Técnicas y Certificación     4333
Fedor Mijailovich Dostoievski                               4296
Graciela Montes                                             3686
Miguel de Cervantes Saavedra                                2746
James Preller                                               2516
Dennis G Zill                                               2497
Jacob Ludwig Karl Grimm                                     2386
Xavier Deneux                                               2332
Andrés Caicedo Estela                                       2326
Jean Paul Sartre                                            2326
Lev Nikolaevich Tolstoi                                     2293
Ernesto Sábato     

## País autor

In [14]:
def get_pais_autor_data_wkd(name, lang):
    template = """
        PREFIX wd: <http://www.wikidata.org/entity/>
        PREFIX wdt: <http://www.wikidata.org/prop/direct/>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

        SELECT ?authorLabel ?countryLabel
        WHERE {{
            BIND("{0}"@{1} AS ?authorLabel) .
            ?author rdfs:label ?authorLabel .
            ?author wdt:P31 wd:Q5 .
            OPTIONAL {{ ?author wdt:P106 wd:Q36180 . }}
            ?author wdt:P19 ?pob .
            ?pob wdt:P17 ?country .
            ?country rdfs:label ?countryLabel .
            FILTER(lang(?countryLabel)="{1}")
        }}
    """
    query = template.format(name, lang)
    url = 'https://query.wikidata.org/bigdata/namespace/wdq/sparql'
    data = requests.get(url, params={'query': query, 'format': 'json'}).json()
    return data

In [15]:
sparql_dbp = SPARQLWrapper("http://dbpedia.org/sparql")

def get_pais_autor_data_dbp(name, lang):
    template = """
        PREFIX dbo: <http://dbpedia.org/ontology/>
        PREFIX dbp: <http://dbpedia.org/property/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

        SELECT ?authorLabel ?countryLabel
        WHERE {{
            BIND("{0}"@{1} AS ?authorLabel) .
            ?author rdfs:label ?authorLabel .
            ?author rdf:type dbo:Person .
            OPTIONAL {{ ?author rdf:type dbo:Writer . }}
            ?author dbp:birthPlace ?pob .
            ?pob rdf:type dbo:Country .
            ?pob rdfs:label ?countryLabel .
            FILTER(lang(?countryLabel)="{1}")
        }}
    """
    query = template.format(name, lang)
    sparql_dbp.setQuery(query)
    sparql_dbp.setReturnFormat(JSON)
    data = sparql_dbp.query().convert()
    return data

In [16]:
def get_pais_autor_data(db, name, lang):
    try:
        if db == 'wdt':
            data = get_pais_autor_data_wkd(name, lang)
        elif db == 'dbp':
            data = get_pais_autor_data_dbp(name, lang)

        results = set(d['countryLabel']['value'] for d in data['results']['bindings'])
        if len(results) == 0:
            result = 'UNKOWN'
        elif len(results) == 1:
            result = results.pop()
        else:
            print('MáS DE UN PAíS PARA', name, ':', results)
            result = results.pop()
    except:
        print('ERROR EN SPARQL PARA', name)
        result = 'UNKOWN'
    return result

In [17]:
def choose(opt_list):
    return Counter(opt_list).most_common(1)[0][0]

In [18]:
# def get_pais_autor(name):
#     r = 'DESCONOCIDO'
#     opt_dict = {}
#     for lang in ['en', 'es']:
#         opt_dict[lang] = {}
#         for db in ['dbp', 'wdt']:
#             opt_dict[lang][db] = get_pais_autor_data(db, name, lang)
#         opts = opt_dict[lang].values()
#         if all(map(lambda x: x != 'UNKOWN', opts)):
#             r = choose(opts)
#             break
#     if r == 'DESCONOCIDO':
#         for lang in ['en', 'es']:
#             opts = opt_dict[lang].values()
#             if any(map(lambda x: x != 'UNKOWN', opts)):
#                 opts = [opt for opt in opts if opt != 'UNKOWN']
#                 r = choose(opts)
#                 break
#     return r

def get_pais_autor(name):
    for lang in ['en', 'es']:
        for db in ['dbp', 'wdt']:
            r = get_pais_autor_data(db, name, lang)
            if r != 'UNKOWN':
                return r
    return 'DESCONOCIDO'

In [19]:
%%time

autores_paises_dict = {}

for i, x in autores_df.drop_duplicates(['AUTOR']).iterrows():
    autores_paises_dict[x['AUTOR']] = x['PAIS_AUTOR']

CPU times: user 19 s, sys: 12.1 ms, total: 19 s
Wall time: 19 s


In [20]:
def combinaciones(ls):
    r = []
    for x in ls:
        aux = list(ls)
        aux.remove(x)
        r.append(aux)
    return r

In [21]:
%%time

for autor in top_autores_desconocidos_sr.index:
    nombres = autor.split()
    if len(nombres) > 1:
        for c in combinaciones(nombres):
            aux = ' '.join(c)
            pais = get_pais_autor(aux)
            if pais != 'DESCONOCIDO':
                autores_paises_dict[autor] = pais
                break

MáS DE UN PAíS PARA James : {'Bangladesh', 'United States of America', 'India', 'Israel'}
MáS DE UN PAíS PARA Roberto Hernández : {'Dominican Republic', 'United States of America', 'Cuba', 'Chile', 'Mexico'}
MáS DE UN PAíS PARA Ricardo : {'Spain', 'Portugal', 'Brazil', 'Chile'}
MáS DE UN PAíS PARA Jacques Rousseau : {'France', 'South Africa'}
MáS DE UN PAíS PARA Diana : {'Brazil', 'Netherlands'}
MáS DE UN PAíS PARA Daniel : {'Brazil', 'Montenegro', 'Russia'}
MáS DE UN PAíS PARA Hugo Gutiérrez : {'Argentina', 'Chile'}
MáS DE UN PAíS PARA Freeman : {'Finland', 'France'}
MáS DE UN PAíS PARA José Silva : {'United States of America', 'Portugal', 'Mexico'}
MáS DE UN PAíS PARA Amber : {'Malta', 'Netherlands'}
MáS DE UN PAíS PARA George Thomas : {'United States of America', 'United Kingdom', 'India', 'Ireland'}
MáS DE UN PAíS PARA Luis Aguilar : {'United States of America', 'Mexico'}
MáS DE UN PAíS PARA Wayne Dyer : {'United States of America', 'United Kingdom'}
MáS DE UN PAíS PARA Bruno : {'I

In [22]:
%%time

def process_pais(x):
    return autores_paises_dict[x['AUTOR']]

autores_df['PAIS_AUTOR'] = autores_df.apply(process_pais, axis=1)

CPU times: user 1min 31s, sys: 59.2 ms, total: 1min 31s
Wall time: 1min 31s


# Saving results

In [23]:
autores_df.to_csv('autores4.csv', index=False)